### Notebook for creating image to image autoencoders for sketches using pytorch

We're using python 3 plus the latest versions of all the packages listed below. Be sure to update before running this nb

In [10]:
### import package

import sys
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy.stats as stats
import pandas as pd
import torch
from PIL import Image
from IPython.display import clear_output
import svgpathtools
import os.path

In [11]:

# directory & file hierarchy
proj_dir = os.path.abspath('..')
code_dir = os.getcwd()
plot_dir = os.path.join(proj_dir,'plots')
data_dir = os.path.join(proj_dir,'data')


if not os.path.exists(code_dir):
    os.makedirs(code_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
    
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [12]:
D = pd.read_csv(os.path.join(data_dir,'semantic_parts_annotated_data.csv'))